In [ ]:
pip install dash

In [ ]:
pip install dash-leaflet

In [1]:
from dash import Dash, dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px
import dash_leaflet as dl


In [ ]:
pip install --upgrade dash jupyter-dash


Function definition (to be removed, just 1 in the code)

In [2]:
import requests
import pandas as pd
import geopandas as gpd
from shapely import wkt
import ipywidgets as widgets
import matplotlib.pyplot as plt
import folium
from shapely.geometry import mapping
#import leafmap as leafmap
from shapely.geometry import shape

def read_response(t):
    try:
        data = t.json() #This will convert the response to a json object
        return data
    except requests.exceptions.JSONDecodeError:
        print("Risposta non valida JSON!")
        print("Contenuto della risposta:", t.text)
        data = None 

def get_measurement_unit(pollutant):
    t=requests.post(url="http://127.0.0.1:5000/api/units", json={"var_pollutant": pollutant}) #json= data will convert the dictionary to a json object and send it to the server
    data = read_response(t) 
    return data[0]
    




# Dashboard

Populate the dropdown

In [3]:
# Function for both DV 10 and DV 11
def pollutant_dropdown():
    t=requests.get(url="http://127.0.0.1:5000/api/pollutant")
    list_pollutant = read_response(t)
    list = [{'label': pollutant, 'value': pollutant} for pollutant in list_pollutant]
    return list



This first part is just for the layout of the elements

In [10]:
# Get pollutants
t = requests.get(url="http://127.0.0.1:5000/api/pollutant")
list_pollutant = read_response(t)


In [4]:
# contour lines file
import json
with open("../DATA/contour_lines.geojson", "r") as f:
    geojson_data = json.load(f)

print(geojson_data["features"][0])  # 🔹 Mostra la prima feature


{'type': 'Feature', 'properties': {'fid': 1, 'ELEV': 2200.0}, 'geometry': {'type': 'LineString', 'coordinates': [[10.237773054012042, 46.63439685859683], [10.237771002440088, 46.63430688921537], [10.237555801129059, 46.63427234207511], [10.237425411671927, 46.63428339632108]]}}


In [5]:
#ADD colour to geojson based on elevation

def get_color(elev, min_elev=500, max_elev=3000):
    ratio = (elev - min_elev) / (max_elev - min_elev)
    ratio = min(1, max(0, ratio))  # Clamping tra 0 e 1

    if ratio < 0.3:
        # Verde (0,255,0) → Giallo (255,255,0)
        r = int(2 * ratio * 255)
        g = 255
        b = 100
    else:
        # Giallo (255,255,0) → Rosso (255,0,0)
        r = 255
        g = int((1 - 2 * (ratio - 0.5)) * 255)
        b = 100

    return f"#{r:02x}{g:02x}{b:02x}"


# Supponiamo tu abbia geojson_data come dizionario o FeatureCollection
for feature in geojson_data["features"]:
    elev = feature["properties"].get("ELEV", 1000)
    color = get_color(elev)
    feature["properties"]["color"] = color


In [8]:
print(geojson_data["features"][0])

{'type': 'Feature', 'properties': {'fid': 1, 'ELEV': 2200.0, 'color': '#ffa364'}, 'geometry': {'type': 'LineString', 'coordinates': [[10.237773054012042, 46.63439685859683], [10.237771002440088, 46.63430688921537], [10.237555801129059, 46.63427234207511], [10.237425411671927, 46.63428339632108]]}}


In [11]:

import dash
from dash import dcc, html, Output, Input, State
import pandas as pd
import geopandas as gpd
import requests

import plotly.express as px

import branca.colormap as cm
import dash_leaflet as dl

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from dash_extensions.javascript import assign
from dash.exceptions import PreventUpdate

def get_years(df):
    return sorted(df["month"].str[-4:].unique())

def get_months(df, year):
    months = sorted(df[df["month"].str.contains(str(year[0]))]["month"].str[:3].unique(),
                    # key to get the months in order
                    key=lambda m: ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"].index(m))
    return ["All"] + months

# --- Dash App ---
app = dash.Dash(__name__)

#DV 10 functions --------------------------------------------------

#from dash_extensions.javascript import assign
# Style function for contour lines (non funziona con Geojson)
def style_function(feature):
    color = feature["properties"].get("color",'blue')  # Default 'blue' se non c'è "color"
    return {
        "color": color,
        "weight": 3,
        "opacity": 0.7
    }  

style = assign("""
function(feature) {
    return {
        color: feature.properties.color || "blue",
        weight: 2,
        opacity: 0.8
    };
}
""")


# LAYOUT 
app.layout = html.Div([
    #DV10
    html.Div([
        html.H3("DV 10 - Correlation map between monthly average of pollutant and station height"),
        
        html.Label("Pollutant:"),
        
        # Dropdown pollutant selection
        dcc.Dropdown(
            id='pollutant-dropdown',
            options=[{'label': p, 'value': p} for p in list_pollutant],
            value=list_pollutant[0]
        ),
        dl.Map(
            id="map",
            center=[45.64, 9.60],
            zoom=8,
            #children=[],
            style={"opacity":0.7, "height": "50vh","position":"relative", 'zIndex':2 },
        ),
    ])
], style={'padding': 20, 'scrollY': 'auto', 'overflowY': 'auto', 'height': '100vh'})  # Imposta lo stile del contenitore principale

# DV 10 callbacks ---------------------------------------------------------------------------
@app.callback(
    Output("map", "children"),
    Input("pollutant-dropdown", "value"),
)
def update_map(selected_pollutant):
    unit = get_measurement_unit(selected_pollutant)
    
    data = {"var_pollutant": selected_pollutant}
    t = requests.post(url="http://127.0.0.1:5000/api/DV_10", json=data)
    data = read_response(t)
    # to visualize the response, we can conver the data to a pandas geodataframe
    gdf = gpd.GeoDataFrame(data)
    gdf['geometry'] = gdf['geometry'].apply(wkt.loads)
    gdf.set_geometry('geometry', inplace=True)

    gdf = gdf.dropna(subset=['quota']) # Drop rows where 'quota' is NaN

    gdf['month'] = pd.to_datetime(gdf['month'], errors='coerce')
    gdf['month'] = gdf['month'].dt.strftime('%b %Y')
    
    
    colormap_c = cm.LinearColormap(["blue", "yellow", "red"],
                                vmin=gdf['monthly_average'].min(),
                                vmax=gdf['monthly_average'].max())
    gdf["color"] = gdf["monthly_average"].apply(lambda x: colormap_c(x)[:7])

    return [
        dl.TileLayer(),  # Base layer 
       
        dl.LayersControl([
            
            # Contour lines (static) (ESCONO SOLO BLU; CON LO STYLE FUNCTION NON FUNZIONANO)
            dl.Overlay(
                dl.GeoJSON(
                        data=geojson_data,
                        zoomToBounds=True,
                        options={"style": style}
                    ),
                    name="Contour lines",
                    checked=True
            ),

            # Overlay dinamici per ogni anno
            *[
                dl.Overlay(
                    dl.LayerGroup([
                        dl.CircleMarker(
                            center=[row.geometry.y, row.geometry.x],
                            radius=5,
                            color=row["color"],
                            fill=False,
                            fillOpacity=0.8,
                            pane="markerPane",
                            children=[dl.Tooltip(f"{row['month']}: {row['monthly_average']} {unit}")]
                        )
                        for _, row in gdf[gdf["month"].str.contains(str(year))].iterrows()
                    ]),
                    name=f"{year}",
                    checked=(year == '2024')  # default most recent year (-1) checked
                )
                for year in get_years(gdf)
            ]
        ])
    ]


# In Jupyter, use run_server not run()
print("Starting Dashboard...")
app.run(port=8089, debug=True)

Starting Dashboard...
